# Imports

In [38]:
import numpy as np
import pandas as pd
import os, json

pd.options.display.max_columns = 30

In [2]:
info = pd.read_csv('Data/superhero_info.csv')
powers = pd.read_csv('Data/superhero_powers.csv')

# Overview

In [3]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.6+ KB


In [5]:
info.head(3)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"


In [6]:
powers.head(3)

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."


# Info Cleanup

In [7]:
info['Hero|Publisher'].str.split('|', expand=True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


In [8]:
info[['Hero','Publisher']] = info['Hero|Publisher'].str.split('|', expand=True)

info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [9]:
info = info.drop(columns='Hero|Publisher')
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [10]:
to_replace = [' cm', ' kg']

for char in to_replace:
    info['Measurements'] = info['Measurements'].str.replace(char,'',
                                                            regex=False)
    
info['Measurements'].head(2)

0    {'Height': '203.0', 'Weight': '441.0'}
1     {'Height': '191.0', 'Weight': '65.0'}
Name: Measurements, dtype: object

In [11]:
info['Measurements'] = info['Measurements'].str.replace("'",'"')

info['Measurements'] = info['Measurements'].apply(json.loads)

info['Measurements'].head(2)

0    {'Height': '203.0', 'Weight': '441.0'}
1     {'Height': '191.0', 'Weight': '65.0'}
Name: Measurements, dtype: object

In [12]:
test_coord = info.loc[0, 'Measurements']
print(type(test_coord))
test_coord

<class 'dict'>


{'Height': '203.0', 'Weight': '441.0'}

In [13]:
height_weight = info['Measurements'].apply(pd.Series)
height_weight

,Height,Weight
0,203.0,441.0
1,191.0,65.0
2,185.0,90.0
3,203.0,441.0
4,193.0,122.0
...,...,...
458,183.0,83.0
459,165.0,52.0
460,66.0,17.0
461,170.0,57.0


In [14]:
info = pd.concat((info, height_weight), axis=1)
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0', 'Weight': '441.0'}",A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0', 'Weight': '65.0'}",Abe Sapien,Dark Horse Comics,191.0,65.0


In [15]:
info = info.drop(columns='Measurements')
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0


# Powers Cleanup

In [16]:
powers.loc[2,'Powers']

'Agility,Accelerated Healing,Cold Resistance,Durability,Underwater breathing,Marksmanship,Weapons Master,Longevity,Intelligence,Super Strength,Telepathy,Stamina,Immortality,Reflexes,Enhanced Sight,Sub-Mariner'

In [17]:
powers['Powers_split'] = '["' + powers['Powers'].astype(str) + '"]'
powers['Powers_split'].head(2)

0       ["Agility,Super Strength,Stamina,Super Speed"]
1    ["Accelerated Healing,Durability,Longevity,Sup...
Name: Powers_split, dtype: object

In [18]:
powers['Powers_split'] = powers['Powers_split'].str.replace(",", '", "')
powers['Powers_split'].head(2)

0    ["Agility", "Super Strength", "Stamina", "Supe...
1    ["Accelerated Healing", "Durability", "Longevi...
Name: Powers_split, dtype: object

In [19]:
powers['Powers_split'] = powers['Powers_split'].apply(json.loads)
powers['Powers_split'].head()

0      [Agility, Super Strength, Stamina, Super Speed]
1    [Accelerated Healing, Durability, Longevity, S...
2    [Agility, Accelerated Healing, Cold Resistance...
3                                 [Lantern Power Ring]
4    [Accelerated Healing, Intelligence, Super Stre...
Name: Powers_split, dtype: object

In [20]:
## exploding the column of lists
exploded = powers.explode('Powers_split')
exploded[['hero_names','Powers', 'Powers_split']].head(5)

,hero_names,Powers,Powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Stamina
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Speed
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Accelerated Healing


In [21]:
cols_to_make = exploded['Powers_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [22]:
for col in cols_to_make:
    powers[col] = powers['Powers'].str.contains(col)
powers.head()

C:\Users\Nemesis Crociata\AppData\Local\Temp\ipykernel_44228\1231346298.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col] = powers['Powers'].str.contains(col)
C:\Users\Nemesis Crociata\AppData\Local\Temp\ipykernel_44228\1231346298.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col] = powers['Powers'].str.contains(col)
C:\Users\Nemesis Crociata\AppData\Local\Temp\ipykernel_44228\1231346298.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

,hero_names,Powers,Powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
powers = powers.drop(columns=['Powers','Powers_split'])

powers.head(2)

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


# Preparing for SQL

In [24]:
name_ids = sorted(powers['hero_names'].unique())
name_ids[:3]

['3-D Man', 'A-Bomb', 'Abe Sapien']

In [25]:
int_ids = range(len(name_ids))
int_ids

range(0, 667)

In [26]:
id_map = dict(zip(name_ids,int_ids))
id_map

{'3-D Man': 0,
 'A-Bomb': 1,
 'Abe Sapien': 2,
 'Abin Sur': 3,
 'Abomination': 4,
 'Abraxas': 5,
 'Absorbing Man': 6,
 'Adam Monroe': 7,
 'Adam Strange': 8,
 'Agent Bob': 9,
 'Agent Zero': 10,
 'Air-Walker': 11,
 'Ajax': 12,
 'Alan Scott': 13,
 'Alex Mercer': 14,
 'Alex Woolsly': 15,
 'Alien': 16,
 'Allan Quatermain': 17,
 'Amazo': 18,
 'Ammo': 19,
 'Ando Masahashi': 20,
 'Angel': 21,
 'Angel Dust': 22,
 'Angel Salvadore': 23,
 'Angela': 24,
 'Animal Man': 25,
 'Annihilus': 26,
 'Ant-Man': 27,
 'Ant-Man II': 28,
 'Anti-Monitor': 29,
 'Anti-Spawn': 30,
 'Anti-Venom': 31,
 'Apocalypse': 32,
 'Aquababy': 33,
 'Aqualad': 34,
 'Aquaman': 35,
 'Arachne': 36,
 'Archangel': 37,
 'Arclight': 38,
 'Ardina': 39,
 'Ares': 40,
 'Ariel': 41,
 'Armor': 42,
 'Astro Boy': 43,
 'Atlas': 44,
 'Atom': 45,
 'Atom Girl': 46,
 'Atom II': 47,
 'Atom III': 48,
 'Atom IV': 49,
 'Aurora': 50,
 'Azazel': 51,
 'Azrael': 52,
 'Aztar': 53,
 'Bananaman': 54,
 'Bane': 55,
 'Banshee': 56,
 'Bantam': 57,
 'Batgirl': 58,

In [27]:
powers['hero_names'].map(id_map)

0        0
1        1
2        2
3        3
4        4
      ... 
662    662
663    663
664    664
665    665
666    666
Name: hero_names, Length: 667, dtype: int64

In [28]:
powers['hero_names'] = powers['hero_names'].replace(id_map)
powers.head(3)

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,0,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,2,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
info['Hero'].head(10)

0           A-Bomb
1       Abe Sapien
2         Abin Sur
3      Abomination
4    Absorbing Man
5     Adam Strange
6        Agent Bob
7       Agent Zero
8       Air-Walker
9             Ajax
Name: Hero, dtype: object

In [31]:
info['Hero'].map(id_map)

0        1
1        2
2        3
3        4
4        6
      ... 
458    661
459    662
460    664
461    665
462    666
Name: Hero, Length: 463, dtype: int64

In [32]:
info['id'] = info['Hero'].replace(id_map)
info.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,id
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,1
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,2
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,3


In [29]:
id_lookup = pd.DataFrame({'str_id': id_map.keys(),
                         'int_id':id_map.values()})
id_lookup.head(3)

,str_id,int_id
0,3-D Man,0
1,A-Bomb,1
2,Abe Sapien,2


In [39]:
merged = pd.merge(info, powers, left_on='id', right_on='hero_names')
merged.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,id,hero_names,Agility,Super Strength,Stamina,...,Insanity,Photographic Reflexes,Anti-Gravity,Thirstokinesis,Power Nullifier,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,1,1,False,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,2,2,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,3,3,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [40]:
merged = merged.drop(columns=['id', 'hero_names'])
merged.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,...,Insanity,Photographic Reflexes,Anti-Gravity,Thirstokinesis,Power Nullifier,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [41]:
cols = merged.columns.tolist()
cols.insert(0, cols.pop(cols.index('Hero')))
df = merged.reindex(columns=cols)

df.head(2)

,Hero,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,Weight,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,...,Insanity,Photographic Reflexes,Anti-Gravity,Thirstokinesis,Power Nullifier,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,A-Bomb,Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Male,Icthyo Sapien,good,No Hair,blue,blue,Dark Horse Comics,191.0,65.0,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [42]:
df.to_csv("Data/final_dataframe.csv.gz", compression='gzip', index=False)

df = pd.read_csv("Data/final_dataframe.csv.gz", low_memory=False)

df.head()

,Hero,Gender,Race,Alignment,Hair color,Eye color,Skin color,Publisher,Height,Weight,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,...,Insanity,Photographic Reflexes,Anti-Gravity,Thirstokinesis,Power Nullifier,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,A-Bomb,Male,Human,good,No Hair,yellow,Unknown,Marvel Comics,203.0,441.0,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Male,Icthyo Sapien,good,No Hair,blue,blue,Dark Horse Comics,191.0,65.0,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Abin Sur,Male,Ungaran,good,No Hair,blue,red,DC Comics,185.0,90.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Abomination,Male,Human / Radiation,bad,No Hair,green,Unknown,Marvel Comics,203.0,441.0,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Absorbing Man,Male,Human,bad,No Hair,blue,Unknown,Marvel Comics,193.0,122.0,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Compare the average weight of super powers who have Super Speed to those who do not.

In [46]:
ss_df = df.groupby('Super Speed')['Weight'].mean()
print(ss_df)

Super Speed
False    101.773585
True     129.404040
Name: Weight, dtype: float64


- The average weight is 101.77 kg for superheroes who *do not* have Super Speed.

- The average weight is 129.40 kg for superheroes who *do* have Super Speed.

## What is the average height of heroes for each publisher?

In [47]:
height_df = df.groupby('Publisher')['Height'].mean()
print(height_df)

Publisher
DC Comics            181.923913
Dark Horse Comics    176.909091
George Lucas         159.600000
Image Comics         211.000000
Marvel Comics        191.546128
Shueisha             171.500000
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Name: Height, dtype: float64


- The values shown are in centimeters.